## Setup

In [1]:
# Setup path
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

print("✅ Setup complete")

✅ Setup complete


## 1. Load Data

In [3]:
from core.multi_asset_loader import load_assets

print("📊 Loading test data...\n")

# Load data for multiple assets
prices = load_assets(
    tickers=['ES', 'GC', 'NQ'],  # 4 assets for testing
    start_date='2015-01-01',
    end_date='2024-12-31'
)

print(f"\n✅ Data loaded:")
for ticker, df in prices.items():
    print(f"   {ticker}: {len(df)} days ({df.index[0].date()} to {df.index[-1].date()})")

📊 Loading test data...


Loading 3 assets: ['ES', 'GC', 'NQ']
  📡 CSV data is 331 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-27)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-27
  📡 CSV data is 331 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-27)
✓ GC: 6334 rows, 2000-08-30 to 2025-11-27
  📡 CSV data is 331 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-27)
✓ NQ: 6362 rows, 2000-09-18 to 2025-11-27

Common date range: 2000-09-18 to 2025-11-27
Filtered date range: 2015-01-01 to 2024-12-31

Total unique dates: 2515
  GC: 2 missing dates, filled 2, 0 remain

ALIGNMENT SUMMARY
ES: 2515/2515 valid dates (100.0%)
GC: 2515/2515 valid dates (100.0%)
NQ: 2515/2515 valid dates (100.0%)

✅ Data loaded:
   ES: 2515 days (2015-01-02 to 2024-12-31)
   GC: 2515 days (2015-01-02 to 2024-12-31)
   NQ: 2515 days (2015-01-02 to 2024-12-31)


## 2. Generate Signals for Strategy 1: Momentum (Fast)

Fast momentum strategy with 60-day lookback

In [4]:
from core.multi_asset_signal import SingleAssetWrapper
from signals.momentum import MomentumSignalV2

print("🎯 Generating STRATEGY 1 signals (Fast Momentum)...\n")

# Strategy 1: Fast Momentum (60-day)
signal_gen_1 = MomentumSignalV2(lookback=60, sma_filter=150)
multi_signal_1 = SingleAssetWrapper(signal_gen_1)
signals_strategy1 = multi_signal_1.generate(prices)

print("Strategy 1 (Fast Momentum - 60 day):")
for ticker, sig_df in signals_strategy1.items():
    long_days = (sig_df['Signal'] == 1).sum()
    total_days = len(sig_df)
    print(f"   {ticker}: {long_days}/{total_days} long days ({long_days/total_days*100:.1f}%)")

🎯 Generating STRATEGY 1 signals (Fast Momentum)...

Strategy 1 (Fast Momentum - 60 day):
   ES: 2293/2515 long days (91.2%)
   GC: 2320/2515 long days (92.2%)
   NQ: 2345/2515 long days (93.2%)


## 3. Generate Signals for Strategy 2: Momentum (Slow)

Slower momentum strategy with 120-day lookback

In [5]:
print("🎯 Generating STRATEGY 2 signals (Slow Momentum)...\n")

# Strategy 2: Slow Momentum (120-day)
signal_gen_2 = MomentumSignalV2(lookback=120, sma_filter=200)
multi_signal_2 = SingleAssetWrapper(signal_gen_2)
signals_strategy2 = multi_signal_2.generate(prices)

print("Strategy 2 (Slow Momentum - 120 day):")
for ticker, sig_df in signals_strategy2.items():
    long_days = (sig_df['Signal'] == 1).sum()
    total_days = len(sig_df)
    print(f"   {ticker}: {long_days}/{total_days} long days ({long_days/total_days*100:.1f}%)")

🎯 Generating STRATEGY 2 signals (Slow Momentum)...

Strategy 2 (Slow Momentum - 120 day):
   ES: 2216/2515 long days (88.1%)
   GC: 2241/2515 long days (89.1%)
   NQ: 2295/2515 long days (91.3%)


## 4. Run Strategy 1 with Allocated Capital

Allocate $60,000 to Strategy 1 (Fast Momentum)

In [ ]:
from core.portfolio import PortfolioManagerV2
import time

print("🚀 Running STRATEGY 1 (Fast Momentum)...\n")

# Strategy 1: Allocated $60K
strategy1_capital = 60000

pm_strategy1 = PortfolioManagerV2(
    initial_capital=strategy1_capital,
    risk_per_trade=0.02,
    max_position_size=0.25,  # 25% per position (4 assets max)
    transaction_cost_bps=3.0,
    slippage_bps=2.0,
    stop_loss_pct=None,
    take_profit_pct=None,
    rebalance_threshold=0.10,
    rebalance_frequency='never'
)

print(pm_strategy1.get_config_summary())

start = time.time()
result_strategy1 = pm_strategy1.run_backtest(signals_strategy1, prices)
duration1 = time.time() - start

print(f"⏱️  Execution time: {duration1:.3f} seconds")
print(f"\n📈 STRATEGY 1 Results:")
result_strategy1.print_summary()

## 5. Run Strategy 2 with Allocated Capital

Allocate $40,000 to Strategy 2 (Slow Momentum)

In [ ]:
print("🚀 Running STRATEGY 2 (Slow Momentum)...\n")

# Strategy 2: Allocated $40K
strategy2_capital = 40000

pm_strategy2 = PortfolioManagerV2(
    initial_capital=strategy2_capital,
    risk_per_trade=0.02,
    max_position_size=0.25,  # 25% per position (4 assets max)
    transaction_cost_bps=3.0,
    slippage_bps=2.0,
    stop_loss_pct=None,
    take_profit_pct=None,
    rebalance_threshold=0.10,
    rebalance_frequency='never'
)

print(pm_strategy2.get_config_summary())

start = time.time()
result_strategy2 = pm_strategy2.run_backtest(signals_strategy2, prices)
duration2 = time.time() - start

print(f"⏱️  Execution time: {duration2:.3f} seconds")
print(f"\n📈 STRATEGY 2 Results:")
result_strategy2.print_summary()

## 6. Combined Portfolio Analysis

Aggregate performance across both strategies

In [ ]:
print("="*80)
print("MULTI-STRATEGY PORTFOLIO SUMMARY")
print("="*80)

# Calculate total portfolio metrics
total_initial_capital = strategy1_capital + strategy2_capital
total_final_equity = result_strategy1.final_equity + result_strategy2.final_equity
total_return = (total_final_equity - total_initial_capital) / total_initial_capital

print(f"\n💼 Portfolio Allocation:")
print(f"   Total Capital:        ${total_initial_capital:,.2f}")
print(f"   Strategy 1 (Fast):    ${strategy1_capital:,.2f} ({strategy1_capital/total_initial_capital:.1%})")
print(f"   Strategy 2 (Slow):    ${strategy2_capital:,.2f} ({strategy2_capital/total_initial_capital:.1%})")

print(f"\n📊 Individual Strategy Performance:")
print(f"\n   Strategy 1 (Fast Momentum):")
print(f"      Allocated:         ${strategy1_capital:,.2f}")
print(f"      Final Equity:      ${result_strategy1.final_equity:,.2f}")
print(f"      P&L:               ${result_strategy1.final_equity - strategy1_capital:,.2f}")
print(f"      Return:            {result_strategy1.metrics['Total Return']:.2%}")
print(f"      Sharpe:            {result_strategy1.metrics['Sharpe Ratio']:.2f}")
print(f"      Max DD:            {result_strategy1.metrics['Max Drawdown']:.2%}")
print(f"      Total Trades:      {result_strategy1.metrics['Total Trades']}")

print(f"\n   Strategy 2 (Slow Momentum):")
print(f"      Allocated:         ${strategy2_capital:,.2f}")
print(f"      Final Equity:      ${result_strategy2.final_equity:,.2f}")
print(f"      P&L:               ${result_strategy2.final_equity - strategy2_capital:,.2f}")
print(f"      Return:            {result_strategy2.metrics['Total Return']:.2%}")
print(f"      Sharpe:            {result_strategy2.metrics['Sharpe Ratio']:.2f}")
print(f"      Max DD:            {result_strategy2.metrics['Max Drawdown']:.2%}")
print(f"      Total Trades:      {result_strategy2.metrics['Total Trades']}")

print(f"\n📈 Combined Portfolio Performance:")
print(f"   Total Initial:        ${total_initial_capital:,.2f}")
print(f"   Total Final:          ${total_final_equity:,.2f}")
print(f"   Total P&L:            ${total_final_equity - total_initial_capital:,.2f}")
print(f"   Total Return:         {total_return:.2%}")

print("\n" + "="*80)

## 7. Strategy Comparison Table

In [ ]:
# Create comparison DataFrame
comparison = pd.DataFrame({
    'Strategy 1 (Fast)': [
        f"${strategy1_capital:,.0f}",
        f"${result_strategy1.final_equity:,.0f}",
        f"{result_strategy1.metrics['Total Return']:.2%}",
        f"{result_strategy1.metrics['Sharpe Ratio']:.2f}",
        f"{result_strategy1.metrics['Max Drawdown']:.2%}",
        result_strategy1.metrics['Total Trades']
    ],
    'Strategy 2 (Slow)': [
        f"${strategy2_capital:,.0f}",
        f"${result_strategy2.final_equity:,.0f}",
        f"{result_strategy2.metrics['Total Return']:.2%}",
        f"{result_strategy2.metrics['Sharpe Ratio']:.2f}",
        f"{result_strategy2.metrics['Max Drawdown']:.2%}",
        result_strategy2.metrics['Total Trades']
    ],
    'Combined Portfolio': [
        f"${total_initial_capital:,.0f}",
        f"${total_final_equity:,.0f}",
        f"{total_return:.2%}",
        "N/A",
        "N/A",
        result_strategy1.metrics['Total Trades'] + result_strategy2.metrics['Total Trades']
    ]
}, index=['Initial Capital', 'Final Equity', 'Total Return', 'Sharpe Ratio', 'Max Drawdown', 'Total Trades'])

print("\n" + "="*95)
print("STRATEGY COMPARISON")
print("="*95)
print(comparison.to_string())
print("="*95)

## 8. Combined Equity Curve

Visualize the combined portfolio performance

In [ ]:
# Align equity curves by date
eq1 = result_strategy1.equity_curve.copy()
eq2 = result_strategy2.equity_curve.copy()

# Ensure same index
eq1 = eq1.set_index('Date') if 'Date' in eq1.columns else eq1
eq2 = eq2.set_index('Date') if 'Date' in eq2.columns else eq2

# Combine equity curves
combined_equity = pd.DataFrame({
    'Strategy1': eq1['TotalValue'],
    'Strategy2': eq2['TotalValue'],
    'Combined': eq1['TotalValue'] + eq2['TotalValue']
})

# Plot
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Combined equity curve
ax1 = axes[0, 0]
combined_equity['Combined'].plot(ax=ax1, linewidth=2, color='purple', label='Combined Portfolio')
ax1.axhline(y=total_initial_capital, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
ax1.set_title('Combined Portfolio - Equity Curve', fontsize=12, fontweight='bold')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Plot 2: Individual strategy comparison
ax2 = axes[0, 1]
combined_equity['Strategy1'].plot(ax=ax2, linewidth=2, color='blue', alpha=0.7, label='Strategy 1 (Fast)')
combined_equity['Strategy2'].plot(ax=ax2, linewidth=2, color='green', alpha=0.7, label='Strategy 2 (Slow)')
ax2.set_title('Individual Strategies Comparison', fontsize=12, fontweight='bold')
ax2.set_ylabel('Portfolio Value ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Plot 3: Normalized performance (% return)
ax3 = axes[1, 0]
norm_equity = pd.DataFrame({
    'Strategy 1': (combined_equity['Strategy1'] / strategy1_capital - 1) * 100,
    'Strategy 2': (combined_equity['Strategy2'] / strategy2_capital - 1) * 100,
    'Combined': (combined_equity['Combined'] / total_initial_capital - 1) * 100
})
norm_equity.plot(ax=ax3, linewidth=2, alpha=0.7)
ax3.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax3.set_title('Normalized Returns (%)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Return (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Strategy allocation over time
ax4 = axes[1, 1]
allocation = pd.DataFrame({
    'Strategy 1': combined_equity['Strategy1'] / combined_equity['Combined'] * 100,
    'Strategy 2': combined_equity['Strategy2'] / combined_equity['Combined'] * 100
})
allocation.plot(ax=ax4, linewidth=2, alpha=0.7)
ax4.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
ax4.set_title('Strategy Allocation Over Time', fontsize=12, fontweight='bold')
ax4.set_ylabel('Allocation (%)')
ax4.set_ylim([0, 100])
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualizations complete")

## 9. Risk Metrics by Strategy

In [ ]:
print("="*80)
print("RISK ANALYSIS BY STRATEGY")
print("="*80)

risk1 = result_strategy1.risk_analysis
risk2 = result_strategy2.risk_analysis

print("\n📊 Strategy 1 (Fast Momentum) Risk Metrics:")
print(f"   Max Drawdown:         {risk1['max_drawdown']:.2%}")
print(f"   Volatility (Ann.):    {risk1['volatility']:.2%}")
print(f"   Downside Vol (Ann.):  {risk1['downside_volatility']:.2%}")
print(f"   VaR (95%):            {risk1['var_95']:.2%}")
print(f"   CVaR (95%):           {risk1['cvar_95']:.2%}")
print(f"   Best Day:             {risk1['best_day']:.2%}")
print(f"   Worst Day:            {risk1['worst_day']:.2%}")

print("\n📊 Strategy 2 (Slow Momentum) Risk Metrics:")
print(f"   Max Drawdown:         {risk2['max_drawdown']:.2%}")
print(f"   Volatility (Ann.):    {risk2['volatility']:.2%}")
print(f"   Downside Vol (Ann.):  {risk2['downside_volatility']:.2%}")
print(f"   VaR (95%):            {risk2['var_95']:.2%}")
print(f"   CVaR (95%):           {risk2['cvar_95']:.2%}")
print(f"   Best Day:             {risk2['best_day']:.2%}")
print(f"   Worst Day:            {risk2['worst_day']:.2%}")

print("\n🎯 Risk Comparison:")
if risk1['volatility'] < risk2['volatility']:
    print(f"   Strategy 1 is LESS volatile ({risk1['volatility']:.2%} vs {risk2['volatility']:.2%})")
else:
    print(f"   Strategy 2 is LESS volatile ({risk2['volatility']:.2%} vs {risk1['volatility']:.2%})")
    
if abs(risk1['max_drawdown']) < abs(risk2['max_drawdown']):
    print(f"   Strategy 1 has SMALLER max drawdown ({risk1['max_drawdown']:.2%} vs {risk2['max_drawdown']:.2%})")
else:
    print(f"   Strategy 2 has SMALLER max drawdown ({risk2['max_drawdown']:.2%} vs {risk1['max_drawdown']:.2%})")

print("\n" + "="*80)

## 10. Next Steps

**✅ Completed:**
- Run multiple strategies simultaneously
- Allocate specific capital to each strategy
- Track individual strategy performance
- Report combined portfolio metrics

**⏭️ Future Enhancements:**
1. **Dynamic Allocation**: Algorithm to decide optimal capital allocation between strategies
2. **Correlation Analysis**: Study how strategies complement each other
3. **Rebalancing Between Strategies**: Shift capital based on performance
4. **MetaPortfolioManager**: Unified class to manage multiple strategies
5. **Risk Parity**: Allocate based on risk contribution rather than fixed amounts